# **Day 9: Mirage Maintenance**

# Setup
The cells below will set up the rest of the notebook. 

I'll start by configuring my kernel:

In [1]:
# Changing the current working directory
%cd ..

# Enabling the autoreload extension
%load_ext autoreload
%autoreload 2

d:\data\programming\advent-of-code-2023


Now, I'm going to import some libraries:

In [2]:
# Import statements
import pandas as pd
import re
from tqdm import tqdm

Finally, I'll load in the data for this puzzle. 

In [3]:
# Load in the data for the puzzle
day = 9
input_data_path = f"data/input-files/day-{day:02d}-input.txt"
example_data_path = f"data/example-input/day-{day:02d}-example.txt"
with open(input_data_path, "r") as txt_file:
    input_data = txt_file.readlines()
input_data = [[int(num) for num in line.strip().split(" ")] for line in input_data]

# Calculating Missing Sequence Numbers
I think that a recursive method could be used to calculate the differences. 

In [6]:
def identify_difference_rows(initial_sequence):
    """
    This helper method will identify the "difference rows" for each of the sequences.
    """

    # First, check if the initial_sequence isn't all 0's
    if all([x == 0 for x in initial_sequence]):
        return None

    # Otherwise, we're going to create a list of the differences
    else:
        diff_list = [
            val - initial_sequence[idx - 1]
            for idx, val in enumerate(initial_sequence)
            if idx > 0
        ]
        remaining_diffs = identify_difference_rows(diff_list)
        my_list = [diff_list]
        if remaining_diffs is not None:
            my_list = [diff_list] + remaining_diffs
        return my_list

Now, with this method in hand, we can calculate the rows for each of the different histories:

In [7]:
history_diff_sequences = {}
for idx, initial_sequence in enumerate(input_data):
    if all([x == 0 for x in initial_sequence]):
        history_diff_sequences[idx] = [initial_sequence]
    else:
        history_diff_sequences[idx] = [initial_sequence] + identify_difference_rows(initial_sequence)

I'll also need a method to find the new numbers: 

In [8]:
def identify_next_sequence_num(diff_sequence):
    """
    This helper method will assist in finding the next number in the sequence. 
    """
    next_sequence_num = 0
    new_cols = [next_sequence_num]
    for row in reversed(diff_sequence):
        next_sequence_num += row[-1]
        new_cols.append(next_sequence_num)
    return next_sequence_num, new_cols[::-1]

Let's put it all together to find the sum of all of the next numbers:

In [9]:
next_numbers_df_records = []
for row_num, diff_sequence in tqdm(list(history_diff_sequences.items())):
    next_sequence_num, new_cols = identify_next_sequence_num(diff_sequence)
    next_numbers_df_records.append(
        {
            "history_num": row_num,
            "history": diff_sequence[0],
            "history_length": len(diff_sequence[0]),
            "diff_sequence": diff_sequence,
            "next_number": next_sequence_num,
            "new_diff_sequence": [
                row + [new_cols[idx]] for idx, row in enumerate(diff_sequence)
            ],
        }
    )
next_numbers_df = pd.DataFrame(next_numbers_df_records)

# Print some of this information
print(f"The sum of the new numbers in each of the histories is '{next_numbers_df['next_number'].sum()}'.")

next_numbers_df.head(5)

100%|██████████| 200/200 [00:00<00:00, 132563.34it/s]

The sum of the new numbers in each of the histories is '1725987467'.


,history_num,history,history_length,diff_sequence,next_number,new_diff_sequence
0,0,"[18, 38, 67, 109, 184, 355, 767, 1698, 3622, 7...",21,"[[18, 38, 67, 109, 184, 355, 767, 1698, 3622, ...",1303999,"[[18, 38, 67, 109, 184, 355, 767, 1698, 3622, ..."
1,1,"[7, 1, -5, 0, 28, 99, 260, 616, 1373, 2893, 57...",21,"[[7, 1, -5, 0, 28, 99, 260, 616, 1373, 2893, 5...",814107,"[[7, 1, -5, 0, 28, 99, 260, 616, 1373, 2893, 5..."
2,2,"[10, 9, 2, -15, -46, -95, -166, -263, -390, -5...",21,"[[10, 9, 2, -15, -46, -95, -166, -263, -390, -...",-6591,"[[10, 9, 2, -15, -46, -95, -166, -263, -390, -..."
3,3,"[20, 32, 46, 66, 96, 140, 202, 286, 396, 536, ...",21,"[[20, 32, 46, 66, 96, 140, 202, 286, 396, 536,...",6012,"[[20, 32, 46, 66, 96, 140, 202, 286, 396, 536,..."
4,4,"[-1, -2, -2, 15, 92, 314, 825, 1846, 3693, 679...",21,"[[-1, -2, -2, 15, 92, 314, 825, 1846, 3693, 67...",480546,"[[-1, -2, -2, 15, 92, 314, 825, 1846, 3693, 67..."


# PART 2
I got this one spoiled for me as I was trawling the Reddit for a hint to why my thing wasn't completing for Part 1 😔 I just need to reverse the lists. 

Part 1 wasn't completing, since I was trying to be clever with the termination of my recursion. (I was looking for "the sum of the history is 0", but I should've been checking all of the values.)

In [12]:
history_diff_sequences = {}
for idx, initial_sequence in enumerate(input_data):
    initial_sequence = initial_sequence[::-1]
    if all([x == 0 for x in initial_sequence]):
        history_diff_sequences[idx] = [initial_sequence]
    else:
        history_diff_sequences[idx] = [initial_sequence] + identify_difference_rows(initial_sequence)

In [13]:
next_numbers_df_records = []
for row_num, diff_sequence in tqdm(list(history_diff_sequences.items())):
    next_sequence_num, new_cols = identify_next_sequence_num(diff_sequence)
    next_numbers_df_records.append(
        {
            "history_num": row_num,
            "history": diff_sequence[0],
            "history_length": len(diff_sequence[0]),
            "diff_sequence": diff_sequence,
            "next_number": next_sequence_num,
            "new_diff_sequence": [
                row + [new_cols[idx]] for idx, row in enumerate(diff_sequence)
            ],
        }
    )
next_numbers_df = pd.DataFrame(next_numbers_df_records)

# Print some of this information
print(f"The sum of the new numbers in each of the histories is '{next_numbers_df['next_number'].sum()}'.")

next_numbers_df.head(5)

100%|██████████| 200/200 [00:00<00:00, 3299.06it/s]

The sum of the new numbers in each of the histories is '971'.


,history_num,history,history_length,diff_sequence,next_number,new_diff_sequence
0,0,"[967936, 707369, 508083, 357970, 246810, 16606...",21,"[[967936, 707369, 508083, 357970, 246810, 1660...",4,"[[967936, 707369, 508083, 357970, 246810, 1660..."
1,1,"[590920, 421408, 294679, 201593, 134563, 87368...",21,"[[590920, 421408, 294679, 201593, 134563, 8736...",8,"[[590920, 421408, 294679, 201593, 134563, 8736..."
2,2,"[-5710, -4911, -4190, -3543, -2966, -2455, -20...",21,"[[-5710, -4911, -4190, -3543, -2966, -2455, -2...",9,"[[-5710, -4911, -4190, -3543, -2966, -2455, -2..."
3,3,"[5200, 4466, 3806, 3216, 2692, 2230, 1826, 147...",21,"[[5200, 4466, 3806, 3216, 2692, 2230, 1826, 14...",6,"[[5200, 4466, 3806, 3216, 2692, 2230, 1826, 14..."
4,4,"[381784, 298603, 229634, 173382, 128299, 92844...",21,"[[381784, 298603, 229634, 173382, 128299, 9284...",0,"[[381784, 298603, 229634, 173382, 128299, 9284..."
